In [2]:
import trafilatura
import requests
import lxml
from elasticsearch import Elasticsearch, helpers


In [3]:
vnexpress = {
    "life": ["https://vnexpress.net/doi-song/cooking",
           "https://vnexpress.net/doi-song/to-am",
           "https://vnexpress.net/doi-song/bai-hoc-song",
           "https://vnexpress.net/doi-song/nha",
           "https://vnexpress.net/doi-song/tieu-dung"],
    "news": ["https://vnexpress.net/thoi-su/chinh-tri",
           "https://vnexpress.net/thoi-su/dan-sinh",
           "https://vnexpress.net/thoi-su/giao-thong",
           "https://vnexpress.net/thoi-su/mekong",
           "https://vnexpress.net/thoi-su/quy-hy-vong"],
    "world": ["https://vnexpress.net/the-gioi/quan-su",
            "https://vnexpress.net/the-gioi/tu-lieu",
            "https://vnexpress.net/the-gioi/phan-tich",
            "https://vnexpress.net/the-gioi/nguoi-viet-5-chau",
            "https://vnexpress.net/the-gioi/cuoc-song-do-day"]
}


In [ ]:
vnexpressLinks = {
    "life": [],
    "news": [],
    "world": []
}
for key, value in vnexpress.items():
    for url in value:
        response = requests.get(url)
        tree = lxml.html.fromstring(response.content)
        title_elem = tree.cssselect('.title-news a')
        for elem in title_elem:
            vnexpressLinks[key].append(elem.get('href'))

print(vnexpressLinks)

In [11]:
vnexpressContent = {
    "life": [],
    "news": [],
    "world": []
}
for key, value in vnexpressLinks.items():
    for url in value:
        if(url.startswith('https://vnexpress.net/')):
            id = url.split('-')[-1].replace('.html', '')
            downloaded = trafilatura.fetch_url(url)
            tree = lxml.html.fromstring(str(downloaded))
            date_elem = len(tree.cssselect(".date")) and tree.cssselect(".date")[0].text or ""
            content = trafilatura.extract(downloaded)
            content = content.replace('\n', ' ')
            item = {"domain": "https://vnexpress.net", "URL": url,"content": content, "date": date_elem,"id": id}
            vnexpressContent[key].append(item)

print(len(vnexpressContent['life'])+len(vnexpressContent['news'])+len(vnexpressContent['world']))

ERROR:trafilatura.core:not enough text None
ERROR:trafilatura.core:not enough text None
ERROR:trafilatura.core:not enough text None
ERROR:trafilatura.core:not enough text None
ERROR:trafilatura.core:not enough text None
ERROR:trafilatura.core:not enough text None
ERROR:trafilatura.core:not enough text None
ERROR:trafilatura.core:not enough text None
ERROR:trafilatura.core:not enough text None


410


In [ ]:
client = Elasticsearch("http://localhost:9200")

client.info()


In [ ]:
def gendata():
    for key,value in vnexpressContent.items():
         for item in value:
            yield {
               "_index": "journal",
               "_type": key,
               "_id": item['id'],
               "doc": item
            }  


helpers.bulk(es, gendata())
